In [2]:
import json
import csv
import os

def convert_menu_to_csv(menu_json_path, output_dir):
    """Convert PetPooja menu.json addon data to CSV files"""
    
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Read menu.json
    with open(menu_json_path, 'r') as f:
        menu_data = json.load(f)
    
    # Build attributes mapping for tags
    attributes_map = {
        attr["attributeid"]: attr["attribute"].lower() 
        for attr in menu_data.get("attributes", [])
    }
    
    # Process addon groups
    addon_groups = []
    addon_items = []
    
    for addon_group_data in menu_data.get("addongroups", []):
        # Add addon group
        group = {
            "id": addon_group_data["addongroupid"],
            "name": addon_group_data["addongroup_name"],
            "display_name": addon_group_data["addongroup_name"],
            "priority": int(addon_group_data.get("addongroup_rank", 0)),
            "is_active": addon_group_data["active"] == "1"
        }
        addon_groups.append(group)
        
        # Process addon items for this group
        for addon_item_data in addon_group_data.get("addongroupitems", []):
            # Get tags from attributes
            tags = []
            if addon_item_data.get("attributes"):
                for attr_id in addon_item_data["attributes"].split(","):
                    attr_id = attr_id.strip()
                    if attr_id in attributes_map:
                        tags.append(attributes_map[attr_id])
            
            item = {
                "addon_group_id": addon_group_data["addongroupid"],
                "id": addon_item_data["addonitemid"],
                "name": addon_item_data["addonitem_name"],
                "display_name": addon_item_data["addonitem_name"],
                "price": float(addon_item_data["addonitem_price"]),
                "is_active": addon_item_data["active"] == "1",
                "priority": int(addon_item_data.get("addonitem_rank", 0)),
                "tags": '"{}"'.format(",".join(tags)) if tags else '""'
            }
            addon_items.append(item)
    
    # Write addon_groups.csv
    groups_file = os.path.join(output_dir, "addon_groups.csv")
    with open(groups_file, 'w', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=["id", "name", "display_name", "priority", "is_active"])
        writer.writeheader()
        writer.writerows(addon_groups)
    
    # Write addon_items.csv  
    items_file = os.path.join(output_dir, "addon_items.csv")
    with open(items_file, 'w', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=[
            "addon_group_id", "id", "name", "display_name", 
            "price", "is_active", "priority", "tags"
        ])
        writer.writeheader()
        writer.writerows(addon_items)
    
    print(f"✅ Created {groups_file}")
    print(f"✅ Created {items_file}")
    print(f"Processed {len(addon_groups)} addon groups and {len(addon_items)} addon items")

if __name__ == "__main__":
    # Example usage
    menu_json_path = "/Users/prakharjain/code/aglio_ai/backend/onboarding_data/handcrafted/menu.json"
    output_dir = "/Users/prakharjain/code/aglio_ai/backend/onboarding_data/handcrafted/customisations"
    convert_menu_to_csv(menu_json_path, output_dir)

✅ Created /Users/prakharjain/code/aglio_ai/backend/onboarding_data/handcrafted/customisations/addon_groups.csv
✅ Created /Users/prakharjain/code/aglio_ai/backend/onboarding_data/handcrafted/customisations/addon_items.csv
Processed 6 addon groups and 35 addon items


In [ ]:
import pandas as pd
import json
import sys
from fuzzywuzzy import fuzz, process

def validate_price_match(csv_price, json_price, tolerance_percent=20, tolerance_absolute=50):
    """
    Validate if two prices are similar enough to be considered a match
    
    Args:
        csv_price: Price from menu.csv
        json_price: Price from menu.json  
        tolerance_percent: Percentage tolerance (e.g., 20 = within 20%)
        tolerance_absolute: Absolute difference tolerance (e.g., 50 = within ±50)
    
    Returns:
        bool: True if prices match within tolerance
    """
    try:
        csv_price = float(csv_price)
        json_price = float(json_price)
        
        # Calculate absolute difference
        abs_diff = abs(csv_price - json_price)
        
        # Calculate percentage difference (using average as base)
        avg_price = (csv_price + json_price) / 2
        percent_diff = (abs_diff / avg_price * 100) if avg_price > 0 else 100
        
        # Accept if within either tolerance
        within_percent = percent_diff <= tolerance_percent
        within_absolute = abs_diff <= tolerance_absolute
        
        return within_percent or within_absolute
        
    except (ValueError, TypeError):
        return False

def fuzzy_match_itemids(menu_csv_path, menu_json_path, output_csv_path=None, name_threshold=70, 
                       price_tolerance_percent=20, price_tolerance_absolute=50, require_price_match=True):
    """
    Match menu.csv items to menu.json items using fuzzy matching with price validation
    
    Args:
        menu_csv_path: Path to menu.csv file
        menu_json_path: Path to menu.json file  
        output_csv_path: Path for output CSV (if None, overwrites input)
        name_threshold: Minimum fuzzy match score for name (0-100)
        price_tolerance_percent: Price tolerance percentage (e.g., 20 = within 20%)
        price_tolerance_absolute: Price tolerance absolute difference
        require_price_match: If True, both name and price must match; if False, name match is sufficient
    """
    
    # Read menu.csv
    print(f"📖 Reading {menu_csv_path}...")
    df_menu = pd.read_csv(menu_csv_path)
    
    # Read menu.json
    print(f"📖 Reading {menu_json_path}...")
    with open(menu_json_path, 'r') as f:
        menu_data = json.load(f)
    
    # Extract items from JSON
    items = menu_data.get("items", [])
    print(f"Found {len(items)} items in menu.json")
    
    # Create lookup dicts
    item_lookup = {item["itemname"]: {"itemid": item["itemid"], "price": item.get("price", "0")} for item in items}
    item_names = list(item_lookup.keys())
    
    print(f"Found {len(df_menu)} rows in menu.csv")
    print(f"🔍 Starting fuzzy matching with price validation...")
    print(f"   Name threshold: {name_threshold}%")
    print(f"   Price tolerance: {price_tolerance_percent}% or ±{price_tolerance_absolute}")
    print(f"   Require price match: {require_price_match}")
    
    # Track matches
    matches_found = 0
    matches_below_name_threshold = 0
    matches_failed_price = 0
    no_matches = 0
    
    # Process each row
    for idx, row in df_menu.iterrows():
        menu_name = row['name']
        menu_price = row.get('price', 0)
        
        # Find best match using fuzzy search
        best_match = process.extractOne(menu_name, item_names, scorer=fuzz.ratio)
        
        if best_match and len(best_match) >= 2:
            matched_name, name_score = best_match[0], best_match[1]
            matched_data = item_lookup[matched_name]
            matched_itemid = matched_data["itemid"]
            matched_price = matched_data["price"]
            
            # Check name threshold
            if name_score >= name_threshold:
                # Validate price if required
                price_match = validate_price_match(
                    menu_price, matched_price, 
                    price_tolerance_percent, price_tolerance_absolute
                )
                
                if not require_price_match or price_match:
                    # Update the id column with itemid
                    df_menu.at[idx, 'id'] = matched_itemid
                    price_status = "✓" if price_match else "⚠"
                    print(f"✅ '{menu_name}' → '{matched_name}' (name: {name_score}%, price: {menu_price}→{matched_price} {price_status}) → ID: {matched_itemid}")
                    matches_found += 1
                else:
                    print(f"💰 '{menu_name}' → '{matched_name}' (name: {name_score}%, price: {menu_price}→{matched_price} ❌) - Price mismatch")
                    matches_failed_price += 1
            else:
                print(f"⚠️  '{menu_name}' → '{matched_name}' (name: {name_score}%, price: {menu_price}→{matched_price}) - Below name threshold")
                matches_below_name_threshold += 1
        else:
            print(f"❌ No match found for '{menu_name}'")
            no_matches += 1
    
    # Save updated CSV
    if output_csv_path is None:
        output_csv_path = menu_csv_path
        
    df_menu.to_csv(output_csv_path, index=False)
    
    print(f"\n📊 Results:")
    print(f"   ✅ Successful matches: {matches_found}")
    print(f"   ⚠️  Below name threshold: {matches_below_name_threshold}")
    print(f"   💰 Failed price validation: {matches_failed_price}")
    print(f"   ❌ No matches found: {no_matches}")
    print(f"   💾 Updated CSV saved to: {output_csv_path}")
    
    return df_menu

def main():
    # if len(sys.argv) < 3:
    #     print("Usage: python fuzzy_match_itemids.py <menu.csv> <menu.json> [output.csv] [threshold]")
    #     print("Example: python fuzzy_match_itemids.py menu.csv sample_petpooja_menu.json updated_menu.csv 70")
    #     return
    
    menu_csv_path = "/Users/prakharjain/code/aglio_ai/backend/onboarding_data/handcrafted/menu.csv"
    menu_json_path = "/Users/prakharjain/code/aglio_ai/backend/onboarding_data/handcrafted/menu.json"
    output_csv_path = "/Users/prakharjain/code/aglio_ai/backend/onboarding_data/handcrafted/menu_updated.csv"
    
    try:
        print("FUZZY MATCHING WITH PRICE VALIDATION")
        print("UPDATE THINGS THAT DIDNT MATCH MANUALLY")
        fuzzy_match_itemids(
            menu_csv_path, 
            menu_json_path, 
            output_csv_path, 
            name_threshold=60,  # Lower name threshold since we have price validation
            price_tolerance_percent=25,  # Allow 25% price difference
            price_tolerance_absolute=100,  # Or ±100 absolute difference
            require_price_match=True  # Require both name and price to match
        )
    except Exception as e:
        print(f"❌ Error: {e}")
        return 1

if __name__ == "__main__":
    main()

📖 Reading /Users/prakharjain/code/aglio_ai/backend/onboarding_data/handcrafted/menu.csv...
📖 Reading /Users/prakharjain/code/aglio_ai/backend/onboarding_data/handcrafted/menu.json...
Found 210 items in menu.json
Found 115 rows in menu.csv

🔍 Starting fuzzy matching...
✅ 'Egg Salad Tartine' → 'Egg Salad Tartine' (score: 100) → ID: 1284214920
✅ 'Greek Salad Tartine' → 'Greek Salad. Tartine' (score: 97) → ID: 1284214919
✅ 'Grilled Veggies & Hummus' → 'Grilled Veggies & Hummus Tartine' (score: 86) → ID: 1284214917
✅ 'Onion Jam & Cheese' → 'Onion Jam & Cheese Tartine' (score: 82) → ID: 1284214918
✅ 'Grilled Chicken Tartine' → 'Grilled Chicken Tartine' (score: 100) → ID: 1284214921
✅ 'Cheesy Corn Balls' → 'Cheesy Corn Balls' (score: 100) → ID: 1284214912
⚠️  'Classic Salted Fries / Wedges' → 'Garlic Cilantro Wedges' (score: 55) - Below threshold, keeping original ID
✅ 'Garlic & Cilantro Fries / Wedges' → 'Garlic & Cilantro Fries' (score: 84) → ID: 1284214906
✅ 'Loaded Nachos' → 'Loaded Nacho

In [1]:
import pandas as pd
import json
import sys

def create_item_addons_csv(menu_csv_path, menu_json_path, output_csv_path):
    """
    Create item_addons.csv by extracting addon relationships from PetPooja menu.json
    
    Args:
        menu_csv_path: Path to menu.csv file
        menu_json_path: Path to menu.json file
        output_csv_path: Path for output item_addons.csv
    """
    
    # Read menu.csv
    print(f"📖 Reading {menu_csv_path}...")
    df_menu = pd.read_csv(menu_csv_path)
    
    # Read menu.json
    print(f"📖 Reading {menu_json_path}...")
    with open(menu_json_path, 'r') as f:
        menu_data = json.load(f)
    
    # Create lookup for PetPooja items by itemid
    petpooja_items = {item["itemid"]: item for item in menu_data.get("items", [])}
    print(f"Found {len(petpooja_items)} items in menu.json")
    
    # Prepare item_addons data
    item_addons = []
    
    print("\n🔗 Processing item-addon relationships...")
    
    for idx, row in df_menu.iterrows():
        menu_item_id = str(row.get("id", "")).strip()
        menu_item_name = row.get("name", "")
        
        if not menu_item_id:
            print(f"⚠️  Skipping row {idx}: No ID found")
            continue
            
        # Find corresponding PetPooja item
        if menu_item_id not in petpooja_items:
            print(f"⚠️  No PetPooja item found for ID: {menu_item_id} ({menu_item_name})")
            continue
            
        petpooja_item = petpooja_items[menu_item_id]
        
        # Check if item allows addons
        if petpooja_item.get("itemallowaddon") != "1":
            print(f"📝 Item '{menu_item_name}' (ID: {menu_item_id}) doesn't allow addons")
            continue
            
        # Process addon relationships
        addon_relationships = petpooja_item.get("addon", [])
        if not addon_relationships:
            print(f"📝 Item '{menu_item_name}' (ID: {menu_item_id}) has no addon groups")
            continue
            
        print(f"✅ Processing '{menu_item_name}' (ID: {menu_item_id}) - {len(addon_relationships)} addon group(s)")
        
        # Create relationships for each addon group
        for priority, addon_data in enumerate(addon_relationships, 1):
            addon_group_id = addon_data.get("addon_group_id")
            min_selection = int(addon_data.get("addon_item_selection_min", 0))
            max_selection = int(addon_data.get("addon_item_selection_max", 1))
            
            # Create unique relationship ID
            relationship_id = f"{menu_item_id}_{addon_group_id}"
            
            item_addon = {
                "id": relationship_id,
                "menu_item_id": menu_item_id,
                "addon_group_id": addon_group_id,
                "min_selection": min_selection,
                "max_selection": max_selection,
                "is_active": "true",
                "priority": priority
            }
            
            item_addons.append(item_addon)
            print(f"   🔗 Added: {addon_group_id} (min:{min_selection}, max:{max_selection})")
    
    # Create DataFrame and save to CSV
    if item_addons:
        df_item_addons = pd.DataFrame(item_addons)
        df_item_addons.to_csv(output_csv_path, index=False)
        print(f"\n📊 Results:")
        print(f"   ✅ Created {len(item_addons)} item-addon relationships")
        print(f"   💾 Saved to: {output_csv_path}")
    else:
        print(f"\n⚠️  No item-addon relationships found to create")
        # Create empty CSV with headers
        df_empty = pd.DataFrame(columns=["id", "menu_item_id", "addon_group_id", "min_selection", "max_selection", "is_active", "priority"])
        df_empty.to_csv(output_csv_path, index=False)
        print(f"   💾 Created empty CSV: {output_csv_path}")
    
    return item_addons

def main():
    # if len(sys.argv) < 4:
    #     print("Usage: python create_item_addons.py <menu.csv> <menu.json> <output_item_addons.csv>")
    #     print("Example: python create_item_addons.py ../../customisations/menu.csv ../../test/sample_petpooja_menu.json item_addons.csv")
    #     return
    
    menu_csv_path = "/Users/prakharjain/code/aglio_ai/backend/onboarding_data/handcrafted/menu.csv"
    menu_json_path = "/Users/prakharjain/code/aglio_ai/backend/onboarding_data/handcrafted/menu.json"
    output_csv_path = "/Users/prakharjain/code/aglio_ai/backend/onboarding_data/handcrafted/item_addons.csv"
    
    try:
        create_item_addons_csv(menu_csv_path, menu_json_path, output_csv_path)
    except Exception as e:
        print(f"❌ Error: {e}")
        return 1

if __name__ == "__main__":
    main()

📖 Reading /Users/prakharjain/code/aglio_ai/backend/onboarding_data/handcrafted/menu.csv...
📖 Reading /Users/prakharjain/code/aglio_ai/backend/onboarding_data/handcrafted/menu.json...
Found 210 items in menu.json

🔗 Processing item-addon relationships...
📝 Item 'Pain Au Chocolate Croissant' (ID: 1284214970) doesn't allow addons
📝 Item 'Twice Baked Almond Croissant' (ID: 1284214969) doesn't allow addons
📝 Item 'Butter Croissant' (ID: 1284214968) doesn't allow addons
📝 Item 'Apple Crumble Tart' (ID: 1284214940) doesn't allow addons
📝 Item 'Lotus Biscoff Cheesecake' (ID: 1284214937) doesn't allow addons
📝 Item 'Blueberry Cheesecake' (ID: 1284214936) doesn't allow addons
📝 Item 'Ginger Ale' (ID: 1284214824) doesn't allow addons
📝 Item 'Lime & Mint Seltzer' (ID: 1284214823) doesn't allow addons
✅ Processing 'Black Forest Mocha Frappe' (ID: 1284214777) - 1 addon group(s)
   🔗 Added: 1560072 (min:1, max:10)
✅ Processing 'Black Forest Frappe' (ID: 1284214776) - 1 addon group(s)
   🔗 Added: 1560